In [25]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
project_folder = '/content/drive/MyDrive/AutoClean'


In [58]:
import os

project_folder = '/content/drive/MyDrive/AutoClean'

# Create necessary directories within Google Drive if not already present
os.makedirs(f'{project_folder}/input', exist_ok=True)
os.makedirs(f'{project_folder}/output', exist_ok=True)
os.makedirs(f'{project_folder}/src', exist_ok=True)
os.makedirs(f'{project_folder}/logs', exist_ok=True)


In [89]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

for file_name in uploaded.keys():
    file_path = f"/content/{file_name}"

    data = pd.read_csv(file_path)

    print(f"File uploaded successfully: {file_name}")


Saving large_dynamic_data_350k.csv to large_dynamic_data_350k.csv
File uploaded successfully: large_dynamic_data_350k.csv


In [90]:
import pandas as pd
import numpy as np
data = pd.read_csv(file_name)

data.head() #printing the first few lines to check if the data has been uploaded correctly

,ID,Category,Value,Date,Text,Age,Salary,City,Score
0,1,A,0.096497,2000-01-01 00:00:00,Lorem,35,148417,Phoenix,87
1,2,A,0.090530,2000-01-01 00:01:00,Amet,70,128069,New York,43
2,3,D,0.047077,2000-01-01 00:02:00,Dolor,65,79396,Chicago,30
3,4,C,0.573380,2000-01-01 00:03:00,Ipsum,27,116692,Houston,60
4,5,B,0.450631,2000-01-01 00:04:00,Dolor,44,77862,Houston,12


In [91]:
missing_values = data.isnull().sum()
print(missing_values)

missing_percentage = data.isnull().mean() * 100

# creating a threshold for missing values to chose whether to drop or replace
threshold = int(input("enter the threshold based on your requirements: "))  # You can adjust this threshold based on your criteria

# iterating through columns to identify and handle the missing data
for col in data.columns:
  if missing_percentage[col] > threshold:
    print(f"Column '{col}' has more than {threshold}% missing values. Dropping rows...")
    data = data.dropna(subset=[col])  # drops rows where that column has NaN values
  else:
    #applying condition to handle numeric and non-numeric values
    if data[col].dtype in ['float64', 'int64']:  # if the column is numeric we will insert the mean
        print(f"Column '{col}' has less than {threshold}% missing values. Filling with mean...")
        data[col] = data[col].fillna(data[col].mean())
    else:
        print(f"Column '{col}' is non-numeric. Dropping rows with missing values...")
        data = data.dropna(subset=[col])  #drop rows that have non-numeric values for their columns

# present cleaned data
print("\nCleaned DataFrame:")
print(data)


ID          0
Category    0
Value       0
Date        0
Text        0
Age         0
Salary      0
City        0
Score       0
dtype: int64
enter the threshold based on your requirements: 10
Column 'ID' has less than 10% missing values. Filling with mean...
Column 'Category' is non-numeric. Dropping rows with missing values...
Column 'Value' has less than 10% missing values. Filling with mean...
Column 'Date' is non-numeric. Dropping rows with missing values...
Column 'Text' is non-numeric. Dropping rows with missing values...
Column 'Age' has less than 10% missing values. Filling with mean...
Column 'Salary' has less than 10% missing values. Filling with mean...
Column 'City' is non-numeric. Dropping rows with missing values...
Column 'Score' has less than 10% missing values. Filling with mean...

Cleaned DataFrame:
            ID Category     Value                 Date   Text  Age  Salary  \
0            1        A  0.096497  2000-01-01 00:00:00  Lorem   35  148417   
1            2  

In [92]:
before_dropping = data.shape[0]
data = data.drop_duplicates()
after_dropping = data.shape[0]
duplicates_removed = before_dropping - after_dropping

print(f"Removed {duplicates_removed} duplicate rows.")


Removed 0 duplicate rows.


In [93]:
# catch only numerical outliers
numeric_data = data.select_dtypes(include=np.number)

Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = (numeric_data < lower_bound) | (numeric_data > upper_bound) #returns boolean dataframe

print("Outliers detected in the following columns:")
print(outliers.any())

data_no_outliers = data.copy()
for col in numeric_data.columns:
    data_no_outliers[col] = np.where(
        data_no_outliers[col] < lower_bound[col],
        lower_bound[col],
        data_no_outliers[col]
    )
    data_no_outliers[col] = np.where(
        data_no_outliers[col] > upper_bound[col],
        upper_bound[col],
        data_no_outliers[col]
    )

print("Outliers capped successfully.")


Outliers detected in the following columns:
ID        False
Value     False
Age       False
Salary    False
Score     False
dtype: bool
Outliers capped successfully.


In [94]:
# Standardising numerical data, mean = 0; std deviation = 1
from sklearn.preprocessing import StandardScaler
import os

base_name = os.path.splitext(os.path.basename(file_path))[0]

standardised_file_name = f"{base_name}_standardised.csv"
output_folder = "/content/drive/My Drive/AutoClean/output"
output_standardised_path = f"{output_folder}/{standardised_file_name}"

scaler = StandardScaler()
data_standardised = data.copy()
data_standardised[data.select_dtypes(include=np.number).columns] = scaler.fit_transform(data.select_dtypes(include=np.number))
print("Data standardised: mean = 0, std = 1")

data_standardised.to_csv(output_standardised_path, index=False)
print(f"Standardised file saved as: {standardised_file_name}")


Data standardised: mean = 0, std = 1
Standardised file saved as: large_dynamic_data_350k_standardised.csv


In [95]:
from sklearn.preprocessing import MinMaxScaler
import os

min_max_scaler = MinMaxScaler()

#Normalize only numeric columns
data_min_max_normalised = data.copy()
data_min_max_normalised[data.select_dtypes(include=np.number).columns] = min_max_scaler.fit_transform(
    data.select_dtypes(include=np.number)
)

base_name = os.path.splitext(os.path.basename(file_path))[0]
min_max_normalised_file_name = f"{base_name}_min_max_normalised.csv"
output_folder = "/content/drive/My Drive/AutoClean/output"
output_min_max_normalised_path = f"{output_folder}/{min_max_normalised_file_name}"

data_min_max_normalised.to_csv(output_min_max_normalised_path, index=False)
print(f"Min-Max Normalized file saved to: {output_min_max_normalised_path}")


Min-Max Normalized file saved to: /content/drive/My Drive/AutoClean/output/large_dynamic_data_350k_min_max_normalised.csv


In [96]:
import os

folder_name = "AutoClean"

folder_path = f"/content/drive/My Drive/{folder_name}"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

cleaned_file_name = f"{base_name}_cleaned.csv"
output_path = os.path.join(folder_path, cleaned_file_name)

print(f"Saving cleaned file to: {output_path}")

data.to_csv(output_path, index=False)

if os.path.exists(output_path):
    print(f"File saved successfully: {output_path}")
else:
    print(f"Failed to save file at {output_path}")


Folder 'AutoClean' already exists.
Saving cleaned file to: /content/drive/My Drive/AutoClean/large_dynamic_data_350k_cleaned.csv
File saved successfully: /content/drive/My Drive/AutoClean/large_dynamic_data_350k_cleaned.csv


In [97]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file_name = f"log_{base_name}_{timestamp}.txt"
log_path = f"{log_folder}/{log_file_name}"

with open(log_path, 'w') as log_file:
    log_file.write(f"Log for file: {base_name}\n")
    log_file.write(f"Timestamp: {timestamp}\n")
    log_file.write(f"Missing value handling: Done\n")
    log_file.write(f"Outlier detection: Done\n")
    log_file.write(f"Duplicate rows removed: {duplicates_removed}\n")
    log_file.write(f"File saved at: {output_path}\n")

print(f"Log saved to: {log_path}")


Log saved to: /content/drive/My Drive/AutoClean/logs/log_large_dynamic_data_350k_2025-04-20_02-08-49.txt
